In [ ]:
# !cd inference && wget https://paddleocr.bj.bcebos.com/dygraph_v2.0/table/en_ppocr_mobile_v2.0_table_det_infer.tar && tar xf en_ppocr_mobile_v2.0_table_det_infer.tar

# !cd inference && wget https://paddleocr.bj.bcebos.com/dygraph_v2.0/table/en_ppocr_mobile_v2.0_table_rec_infer.tar && tar xf en_ppocr_mobile_v2.0_table_rec_infer.tar

# !cd inference && wget https://paddleocr.bj.bcebos.com/ppstructure/models/slanet/paddle3.0b2/en_ppstructure_mobile_v2.0_SLANet_infer.tar && tar xf en_ppstructure_mobile_v2.0_SLANet_infer.tar

# !cd inference && wget https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_det_infer.tar && tar xf en_PP-OCRv3_det_infer.tar

# !cd inference && wget https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_rec_infer.tar && tar xf en_PP-OCRv3_rec_infer.tar


    # !python3 /home/ubuntu/OCR/PaddleOCR/ppstructure/table/predict_table.py \
    #     --det_model_dir=/home/ubuntu/OCR/inference/en_PP-OCRv3_det_infer \
    #     --rec_model_dir=/home/ubuntu/OCR/inference/cyrillic_PP-OCRv3_rec \
    #     --table_model_dir=/home/ubuntu/OCR/inference/en_ppstructure_mobile_v2.0_SLANet_infer \
    #     --rec_char_dict_path=/home/ubuntu/OCR/PaddleOCR/ppocr/utils/dict/cyrillic_dict.txt \
    #     --table_char_dict_path=/home/ubuntu/OCR/PaddleOCR/ppocr/utils/dict/table_structure_dict.txt \
    #     --image_dir=/home/ubuntu/OCR/data/det/train/4.jpg \
    #     --output=/home/ubuntu/OCR/output/table


--2025-04-29 12:13:41--  https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_det_infer.tar
Resolving paddleocr.bj.bcebos.com (paddleocr.bj.bcebos.com)... 103.235.47.176, 2402:2b40:7000:628:0:ff:b0e8:88da
Connecting to paddleocr.bj.bcebos.com (paddleocr.bj.bcebos.com)|103.235.47.176|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4003840 (3.8M) [application/x-tar]
Saving to: ‘en_PP-OCRv3_det_infer.tar’

en_PP-OCRv3_det_inf 100%[===================>]   3.82M   913KB/s    in 15s     

2025-04-29 12:13:57 (267 KB/s) - ‘en_PP-OCRv3_det_infer.tar’ saved [4003840/4003840]



In [5]:
import os
import subprocess
import logging

# Configure logging
logging.basicConfig(
    level=logging.DEBUG,
    format="[%(asctime)s] ppocr %(levelname)s: %(message)s",
    datefmt="%Y/%m/%d %H:%M:%S",
)

# --- NEW: Define the artifact output directory ---
artifact_output_dir = "/home/ubuntu/OCR/output/table/paddle_pipeline"
os.makedirs(artifact_output_dir, exist_ok=True)
logging.info(f"Artifact output directory created: {artifact_output_dir}")

# Set up the paths
det_model_dir = "/home/ubuntu/OCR/inference/en_PP-OCRv3_det_infer"
rec_model_dir = "/home/ubuntu/OCR/inference/cyrillic_PP-OCRv3_rec"
table_model_dir = "/home/ubuntu/OCR/inference/en_ppstructure_mobile_v2.0_SLANet_infer"
rec_char_dict_path = "/home/ubuntu/OCR/PaddleOCR/ppocr/utils/dict/cyrillic_dict.txt"
table_char_dict_path = (
    "/home/ubuntu/OCR/PaddleOCR/ppocr/utils/dict/table_structure_dict.txt"
)
image_path = "/home/ubuntu/OCR/data/det/train/4.jpg"
output_dir = os.path.join(
    artifact_output_dir, "raw_output"
)  # Save raw output here initially
os.makedirs(output_dir, exist_ok=True)
logging.info(f"Raw output directory created: {output_dir}")

# Build the command for predict_table.py - removed --lang=en
cmd = [
    "python3",
    "/home/ubuntu/OCR/PaddleOCR/ppstructure/table/predict_table.py",
    f"--det_model_dir={det_model_dir}",
    f"--rec_model_dir={rec_model_dir}",
    f"--table_model_dir={table_model_dir}",
    f"--rec_char_dict_path={rec_char_dict_path}",
    f"--table_char_dict_path={table_char_dict_path}",
    f"--image_dir={image_path}",
    f"--output={output_dir}",
    "--use_gpu=True",
]

# Run the command and capture output
logging.info("Running predict_table.py through subprocess")
logging.info(" ".join(cmd))

import time

start_time = time.time()

try:
    result = subprocess.run(
        cmd, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True
    )

    # Print stdout and stderr
    logging.info("Command output:")
    logging.info(result.stdout)

    if result.stderr:
        logging.warning("Error output:")
        logging.warning(result.stderr)

    # Check if Excel file exists in the raw output directory
    excel_path = os.path.join(
        output_dir, os.path.basename(image_path).split(".")[0] + ".xlsx"
    )
    if os.path.exists(excel_path):
        logging.info(f"Excel file confirmed at {excel_path}")

        # --- NEW: Move the Excel file to the artifact output directory ---
        artifact_excel_path = os.path.join(
            artifact_output_dir, os.path.basename(excel_path)
        )
        os.rename(excel_path, artifact_excel_path)
        logging.info(f"Excel file moved to: {artifact_excel_path}")

        # --- NEW: Check if the JSON file exists and move it ---
        json_path = os.path.join(
            output_dir, os.path.basename(image_path).split(".")[0] + ".json"
        )
        if os.path.exists(json_path):
            artifact_json_path = os.path.join(
                artifact_output_dir, os.path.basename(json_path)
            )
            os.rename(json_path, artifact_json_path)
            logging.info(f"JSON file moved to: {artifact_json_path}")
        else:
            logging.warning(f"JSON file not found at: {json_path}")

    else:
        logging.error(f"Excel file not found at {excel_path}")

except subprocess.CalledProcessError as e:
    logging.error(f"Command failed with exit code {e.returncode}")
    logging.error(f"Error output: {e.stderr}")

end_time = time.time()
logging.info(f"Total processing time: {end_time - start_time:.3f}s")

[2025/04/29 12:16:39] ppocr INFO: Artifact output directory created: /home/ubuntu/OCR/output/table/paddle_pipeline
[2025/04/29 12:16:39] ppocr INFO: Raw output directory created: /home/ubuntu/OCR/output/table/paddle_pipeline/raw_output
[2025/04/29 12:16:39] ppocr INFO: Running predict_table.py through subprocess
[2025/04/29 12:16:39] ppocr INFO: python3 /home/ubuntu/OCR/PaddleOCR/ppstructure/table/predict_table.py --det_model_dir=/home/ubuntu/OCR/inference/en_PP-OCRv3_det_infer --rec_model_dir=/home/ubuntu/OCR/inference/cyrillic_PP-OCRv3_rec --table_model_dir=/home/ubuntu/OCR/inference/en_ppstructure_mobile_v2.0_SLANet_infer --rec_char_dict_path=/home/ubuntu/OCR/PaddleOCR/ppocr/utils/dict/cyrillic_dict.txt --table_char_dict_path=/home/ubuntu/OCR/PaddleOCR/ppocr/utils/dict/table_structure_dict.txt --image_dir=/home/ubuntu/OCR/data/det/train/4.jpg --output=/home/ubuntu/OCR/output/table/paddle_pipeline/raw_output --use_gpu=True
[2025/04/29 12:16:48] ppocr INFO: Command output:
[2025/04/29

In [2]:
!python3 /home/ubuntu/OCR/PaddleOCR/ppstructure/table/eval_table.py \
    --det_model_dir=/home/ubuntu/OCR/inference/en_PP-OCRv3_det_infer \
    --rec_model_dir=/home/ubuntu/OCR/inference/cyrillic_PP-OCRv3_rec \
    --table_model_dir=/home/ubuntu/OCR/inference/en_ppstructure_mobile_v2.0_SLANet_infer \
    --image_dir=/home/ubuntu/OCR/data/det/train/ \
    --rec_char_dict_path=/home/ubuntu/OCR/PaddleOCR/ppocr/utils/dict/cyrillic_dict.txt \
    --table_char_dict_path=/home/ubuntu/OCR/PaddleOCR/ppocr/utils/dict/table_structure_dict.txt \
    --det_limit_side_len=960 \
    --det_limit_type=max \
    --rec_image_shape=3,48,320 \
    --gt_path=/home/ubuntu/OCR/output/table/gt.txt \
    --process_id=0 \
    --total_process_num=1

/home/ubuntu/ocr_venv_py38/lib/python3.8/site-packages/paddle/utils/cpp_extension/extension_utils.py:711: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)
[2025/04/21 16:37:43] ppocr WARNING: The first GPU is used for inference by default, GPU ID: 0
[2025/04/21 16:37:46] ppocr WARNING: The first GPU is used for inference by default, GPU ID: 0
[2025/04/21 16:37:46] ppocr WARNING: The first GPU is used for inference by default, GPU ID: 0
[2025/04/21 16:37:47] ppocr WARNING: The first GPU is used for inference by default, GPU ID: 0
100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 54.49it/s]
0.00it [00:00, ?it/s]
0it [00:00, ?it/s]
[2025/04/21 16:37:48] ppocr INFO: teds: 0.5167851479254988
